# Datafonos

Uno de nuestros clientes, **CLAP**, maneja Datafonos y se encarga de procesar
pagos. **SIMETRIK** es su aliado para comparar las transacciones que se
reportan a través de sus dispositivos contra las
transacciones que se registran en las liquidaciones
bancarias (**BANSUR**).  
Cada vez que el cliente **CLAP** reporta un pago con su
datáfono envía un registro a su base de datos y a la base
de datos del adquirente **BANSUR**. Sin embargo, dada la
gran cantidad de pagos que se realizan a diario, **CLAP** no puede llevar un
control efectivo de todas las transacciones realizadas y necesitan verificar
que todas las transacciones registradas en su base de datos también se
encuentren en la base de **BANSUR**.

*IMPORTANTE:* Una transacción regular se evidencia en la base de datos
como un **PAGO**; se debe tener en cuenta que un *mismo ID* puede también
tomar estado de **Cancelación**, **Chargeback** u **Otros casos**.  
Simetrik considera una partida como conciliable toda aquella transacción
cuyo último estado en la base de datos ordenada por fecha y hora sea
PAGADA.  
Para esto, **SIMETRIK** comparará para cada transacción campos únicos entre
las dos entidades buscando encontrar parejas que sean exactamente
iguales bajo las siguientes condiciones:  
1. Que tengan el mismo **ID**.
2. Que tenga los mismos 6 primeros dígitos de la tarjeta.
3. Que tengan los mismos 4 últimos dígitos de la tarjeta.
4. Que el valor pagado en la transacción sea igual o que su diferencia
esté en el rango de más o menos 0.99 pesos.
5. Que tengan la misma fecha de transacción.

In [1]:
import pandas as pd

In [3]:
bansur_data = pd.read_csv('data/bansur.csv')
print(bansur_data)

           TARJETA TIPO_TRX    MONTO  FECHA_TRANSACCION CODIGO_AUTORIZACION  \
0       9540150300     PAGO     44.0           20201101              272012   
1       9540150300     PAGO     93.0           20201101              216149   
2       9540150310     PAGO   1579.0           20201101              898544   
3       9540150319     PAGO     86.0           20201101              501928   
4       9540150314     PAGO    679.0           20201101              170207   
...            ...      ...      ...                ...                 ...   
132391  4450167626     PAGO    350.0           20201101              149593   
132392  4427328952     PAGO   5000.0           20201101              053891   
132393  4427537641     PAGO   5000.0           20201101              042185   
132394  4461382609     PAGO  17500.0           20201101              755434   
132395  4461382609     PAGO  50000.0           20201101              751564   

        ID_ADQUIRIENTE FECHA_RECEPCION  
0         

In [4]:
clap_data = pd.read_csv('data/clap.csv')
print(clap_data)

        INICIO6_TARJETA  FINAL4_TARJETA   TIPO_TRX     MONTO  \
0              443715.0             190  CANCELADA  21944.00   
1              802414.0            6615     PAGADA     42.51   
2              818156.0             672  NO APLICA     20.80   
3              954015.0            5581     PAGADA     41.00   
4              954015.0            2538     PAGADA     68.00   
...                 ...             ...        ...       ...   
163544         954015.0            9059     PAGADA    128.00   
163545         404826.0            5173     PAGADA    416.88   
163546         928869.0            8072  CANCELADA     20.00   
163547         922075.0            4913  CANCELADA    373.36   
163548         954015.0            2015  CANCELADA    140.00   

              FECHA_TRANSACCION CODIGO_AUTORIZACION      ID_BANCO  \
0       2020-11-01 07:50:25.000              136719  4.710019e+10   
1       2020-11-01 04:35:04.000              053042  2.960773e+10   
2       2020-11-01 04:35